In [ ]:
# ============================================================
# nn_liml_realdata.py  (Real-data NN first stage + LIML L1)
# - Endogenous: LABOR, COWS, FEED, ROUGHAGE
# - Exogenous in X: CONST, LAND, YEAR dummies (as in the paper)
# - Z: CONST, LAND, PMILK, PFEED, LANDOWN, COOP dummies, YEAR dummies
# - Augments Z up to "C": squares + selected interactions
# - First-stage baselines: OLS and ElasticNetCV
# - PCA for NN first-stage (keep 95% variance), optional for OLS/EN
# - Adds: standardized Z cond number, OPG SEs, TE summary, Σ̂ηη spectrum
# - Variance bounds/penalties relaxed to avoid boundary sticking
# ============================================================

import os
import math
import warnings
import numpy as np
import pandas as pd
from dataclasses import dataclass
from typing import Tuple, Optional, Dict, Any, List

from scipy.stats import norm
from scipy.optimize import minimize
from scipy.linalg import cho_factor, cho_solve

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from sklearn.linear_model import LinearRegression, ElasticNetCV
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold

# ---------------- Global setup ----------------
BASE_SEED = 12345
np.random.seed(BASE_SEED)
torch.manual_seed(BASE_SEED)
pd.set_option('display.float_format', '{:.4f}'.format)

# ---------------- Config toggles ----------------
PCA_Z_FOR_NN = False           # PCA compress Z before NN first-stage
PCA_Z_FOR_OLS_LASSO = False    # Use same PCA-compressed Z in OLS/ElasticNet baselines
PCA_VAR_KEEP = 0.95           # fraction of variance to retain in PCA for NN/baselines

# ==============================
# Utilities
# ==============================
def make_spd_from_symmetric(A: np.ndarray, jitter: float = 1e-8) -> np.ndarray:
    """Nearest-ish SPD: symmetrize + add jitter if needed (checked by Cholesky)."""
    A = 0.5*(A + A.T)
    try:
        cho_factor(A, lower=False, check_finite=False)
        return A
    except Exception:
        bump = max(jitter, 1e-6*(1.0 + float(np.max(np.abs(np.diag(A))) if A.size else 0.0)))
        while True:
            A_try = A + bump*np.eye(A.shape[0])
            try:
                cho_factor(A_try, lower=False, check_finite=False)
                return A_try
            except Exception:
                bump *= 10.0

def standardize_fit(X: np.ndarray) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    mu = X.mean(axis=0, keepdims=True)
    sd = X.std(axis=0, keepdims=True)
    sd = np.where(sd < 1e-8, 1.0, sd)
    return (X - mu)/sd, mu, sd

def standardize_apply(X: np.ndarray, mu: np.ndarray, sd: np.ndarray) -> np.ndarray:
    return (X - mu)/sd

def kfold_indices(n: int, K: int, seed: int = 12345):
    rng = np.random.default_rng(seed)
    idx = np.arange(n)
    rng.shuffle(idx)
    folds = np.array_split(idx, K)
    for k in range(K):
        te = folds[k]
        tr = np.hstack([folds[j] for j in range(K) if j != k])
        yield tr, te

def r2_score(y, yhat):
    sst = np.sum((y - y.mean())**2)
    sse = np.sum((y - yhat)**2)
    return 1.0 - sse / max(sst, 1e-12)

def te_from_r_sigma(r: np.ndarray, sigma_u: float, sigma_c: float) -> np.ndarray:
    """Jondrow et al. E[exp(-u)|ε=r] for half-normal u and Normal v."""
    sigma2 = sigma_u**2 + sigma_c**2
    mu_star = -r * (sigma_u**2) / sigma2
    sigma_star = (sigma_u * sigma_c) / np.sqrt(sigma2)
    a = mu_star / (sigma_star + 1e-15)
    log_ratio = norm.logcdf(a - sigma_star) - norm.logcdf(a)
    return np.exp(-mu_star + 0.5 * sigma_star**2 + log_ratio)

def _round_list(xs, nd=4):
    # Robust to numpy arrays / scalars
    try:
        return [round(float(v), nd) for v in xs]
    except TypeError:
        return [round(float(xs), nd)]


# ==============================
# NN first stage (small-n stable)
# ==============================
@dataclass
class NNConfig:
    epochs: int = 800
    batch_size: int = 32
    lr: float = 2e-4
    weight_decay: float = 1e-3
    hidden: Tuple[int, ...] = (128, 64)
    dropout: float = 0.45
    device: str = "cuda" if torch.cuda.is_available() else "cpu"
    seed: int = BASE_SEED
    K: int = 10

class RF_NN(nn.Module):
    """Compact MLP + learned residual covariance via Cholesky parameter Lpar."""
    def __init__(self, z_dim: int, k2: int, hidden=(128, 64), dropout=0.45):
        super().__init__()
        layers = []
        prev = z_dim
        for h in hidden:
            layers += [
                nn.Linear(prev, h),
                nn.ELU(),
                nn.Dropout(dropout)
            ]
            prev = h
        layers += [nn.Linear(prev, k2)]
        self.mlp = nn.Sequential(*layers)
        self.Lpar = nn.Parameter(torch.eye(k2) * 0.5)

    def _Sigma(self):
        L = torch.tril(self.Lpar)
        d = F.softplus(torch.diag(L)) + 0.5  # keep positive diagonal
        L = L - torch.diag(torch.diag(L)) + torch.diag(d)
        return L @ L.T, L

    def nll(self, Zb, X2b):
        """Gaussian MLE on residuals with full Σ_res = LLᵀ."""
        X2_hat = self.mlp(Zb)
        resid = X2b - X2_hat
        Sigma, L = self._Sigma()
        Y = torch.linalg.solve_triangular(L, resid.T, upper=False)
        quad = (Y**2).sum()
        logdet = 2.0 * torch.log(torch.clamp(torch.diag(L), min=1e-6)).sum()
        m = resid.shape[0]
        reg = 1e-3 * torch.norm(self.Lpar)**2
        return 0.5 * (quad + m * logdet) + reg

def train_one_fold(Ztr, X2tr, Zva, X2va, cfg: NNConfig):
    torch.manual_seed(cfg.seed)
    device = torch.device(cfg.device)
    Ztr_t = torch.tensor(Ztr, dtype=torch.float32, device=device)
    X2tr_t = torch.tensor(X2tr, dtype=torch.float32, device=device)
    Zva_t = torch.tensor(Zva, dtype=torch.float32, device=device)
    X2va_t = torch.tensor(X2va, dtype=torch.float32, device=device)

    model = RF_NN(Ztr.shape[1], X2tr.shape[1], cfg.hidden, cfg.dropout).to(device)
    opt = optim.AdamW(model.parameters(), lr=cfg.lr, weight_decay=cfg.weight_decay)

    def lr_lambda(epoch):
        if epoch < 100:
            return 0.1 + 0.9 * (epoch / 100)
        else:
            return 0.5 * (1 + math.cos(math.pi * (epoch - 100) / (cfg.epochs - 100)))
    scheduler = optim.lr_scheduler.LambdaLR(opt, lr_lambda)

    ntr = Ztr.shape[0]
    steps = max(1, ntr // cfg.batch_size)
    best = np.inf
    best_state = None
    patience, wait = 30, 0

    for epoch in range(cfg.epochs):
        model.train()
        perm = torch.randperm(ntr, device=device)
        for s in range(steps):
            si = s * cfg.batch_size
            ei = min((s+1) * cfg.batch_size, ntr)
            if si >= ei: break
            idx = perm[si:ei]
            loss = model.nll(Ztr_t[idx], X2tr_t[idx])
            opt.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5.0)
            opt.step()

        if epoch % 5 == 0:
            model.eval()
            with torch.no_grad():
                val_loss = model.nll(Zva_t, X2va_t).item()
            if val_loss < best - 1e-6:
                best = val_loss
                best_state = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}
                wait = 0
            else:
                wait += 1
            if wait >= patience:
                break
        scheduler.step()

    if best_state is not None:
        model.load_state_dict(best_state)
    model.eval()
    return model

def shrink_cov(S: np.ndarray, rho: float = 0.15) -> np.ndarray:
    """Ledoit–Wolf style shrinkage to stabilize Σ^{-1} at small n."""
    S = 0.5*(S + S.T)
    k = S.shape[0]
    tau = np.trace(S) / max(k, 1)
    S_shrunk = (1.0 - rho) * S + rho * (tau * np.eye(k))
    return make_spd_from_symmetric(S_shrunk, jitter=1e-8)

def crossfit_nn_l2(Z: np.ndarray, X2: np.ndarray, cfg: NNConfig, rho_shrink: float = 0.15):
    """K-fold cross-fitted NN reduced form + shrunk Σ̂_{ηη}."""
    n, _ = Z.shape
    k2 = X2.shape[1]
    eta_hat = np.zeros((n, k2), dtype=np.float64)

    for (tr, te) in kfold_indices(n, cfg.K, seed=cfg.seed):
        Ztr, Zte = Z[tr], Z[te]
        X2tr, X2te = X2[tr], X2[te]

        Ztr_std, Zmu, Zsd = standardize_fit(Ztr)
        Zte_std = standardize_apply(Zte, Zmu, Zsd)

        X2tr_std, X2mu, X2sd = standardize_fit(X2tr)
        X2te_std = standardize_apply(X2te, X2mu, X2sd)

        ntr = Ztr_std.shape[0]
        nva = max(6, int(0.10 * ntr))
        va_idx = np.arange(ntr)[:nva]
        tr_idx = np.arange(ntr)[nva:]

        model = train_one_fold(
            Ztr_std[tr_idx], X2tr_std[tr_idx],
            Ztr_std[va_idx],  X2tr_std[va_idx],
            cfg
        )

        with torch.no_grad():
            device = torch.device(cfg.device)
            Zte_t = torch.tensor(Zte_std, dtype=torch.float32, device=device)
            X2_hat_te_std = model.mlp(Zte_t).cpu().numpy()

        # back to original scale
        X2_hat_te = X2_hat_te_std * X2sd + X2mu
        eta_hat[te] = X2te - X2_hat_te

    S = np.cov(eta_hat, rowvar=False)
    S = make_spd_from_symmetric(S, jitter=1e-8)
    S_shrunk = shrink_cov(S, rho=rho_shrink)
    return eta_hat, S_shrunk

# ==============================
# LIML L1 (fixed Σ̂ηη from NN)
# ==============================
def lnL1_vec_fixed_sigee(beta, sig_ev, sigee, log_sigma_v, log_lambda_c, y, X, etas):
    """Per-observation log-likelihood for the L1 (ε=v-u) model."""
    sigee = make_spd_from_symmetric(sigee, jitter=1e-10)
    cF = cho_factor(sigee, lower=False, check_finite=False)
    w = cho_solve(cF, sig_ev, check_finite=False)          # w = Σ_{ηη}^{-1} Σ_{ηv}
    muci = etas @ w                                        # control function
    r = y - X @ beta - muci

    sigma_v = np.exp(log_sigma_v)
    lam_c   = np.exp(log_lambda_c)                         # λ_c = σ_u/σ_c
    correction = float(sig_ev @ w)                         # Σ_{vη} Σ_{ηη}^{-1} Σ_{ηv}
    sigma_c2 = sigma_v**2 - correction
    if sigma_c2 <= 1e-12:
        return np.full(y.shape[0], -1e20)                  # impossible region
    sigma_c = np.sqrt(sigma_c2)
    sigma2  = sigma_c2 + (lam_c * sigma_c)**2
    sigma   = np.sqrt(sigma2)

    z = -(lam_c / sigma) * r
    z = np.clip(z, -10, 10)                                # numeric guard
    lPhi = norm.logcdf(z)
    return -0.5*np.log(sigma2) - 0.5*(r**2 / sigma2) + lPhi

def neg_loglik_L1_only_with_penalty(theta, y, X, etas, sigee, k2):
    """Total negative log-likelihood + soft regularization."""
    p = X.shape[1]
    beta = theta[:p]
    sig_ev = theta[p:p+k2]
    log_sigma_v = theta[-2]
    log_lambda_c = theta[-1]

    l1 = lnL1_vec_fixed_sigee(beta, sig_ev, sigee, log_sigma_v, log_lambda_c, y, X, etas)
    val = -float(np.sum(l1))
    if not np.isfinite(val):
        return 1e20

    # feasibility guard
    cF = cho_factor(sigee, lower=False, check_finite=False)
    w = cho_solve(cF, sig_ev, check_finite=False)
    correction = float(sig_ev @ w)
    sigma_v = np.exp(log_sigma_v)
    sigma_c2 = sigma_v**2 - correction
    if sigma_c2 <= 1e-12:
        return 1e20

    # softer priors
    sigma_v_center  = 0.05 * (log_sigma_v - 0.0)**2     # gentle center at log(1)
    lambda_c_center = 0.10 * (log_lambda_c - np.log(0.3))**2
    impossible_penalty = 1e3 * max(0, -sigma_c2)
    param_penalty = 0.02 * (np.sum(beta**2) + np.sum(sig_ev**2))

    return val + sigma_v_center + lambda_c_center + impossible_penalty + param_penalty

def estimate_LIML_L1_only(y, X, etas, sigee, fallback=True):
    """Optimize the L1 likelihood for (β, Σ_{ηv}, σ_v, λ_c) given Σ̂_{ηη}."""
    n, p = X.shape
    k2   = etas.shape[1]

    beta0 = np.linalg.lstsq(X, y, rcond=1e-8)[0]
    e0 = y - X @ beta0
    try:
        sig_ev0 = np.linalg.lstsq(etas, e0, rcond=1e-8)[0]
    except np.linalg.LinAlgError:
        sig_ev0 = np.zeros(k2)

    # multiple starts for variance params
    starts_sigma_v = [np.log(1.0), np.log(0.6), np.log(1.5), np.log(2.0)]
    starts_lambda_c = [np.log(0.3), np.log(0.5), np.log(0.1), np.log(0.8)]

    # WIDER bounds, allow σ_v down to 0.10
    lower = [-np.inf]*p + [-5.0]*k2 + [np.log(0.10), np.log(0.02)]
    upper = [ np.inf]*p + [ 5.0]*k2 + [np.log(5.00), np.log(3.00)]
    bounds = list(zip(lower, upper))

    theta0_list = []
    for sv in starts_sigma_v:
        for lc in starts_lambda_c:
            theta0_list.append(np.concatenate([beta0, sig_ev0, [sv, lc]]))

    methods = ['L-BFGS-B'] + (['SLSQP'] if fallback else [])
    best_res, best_fun = None, np.inf
    for theta0 in theta0_list:
        for m in methods:
            try:
                res = minimize(neg_loglik_L1_only_with_penalty, theta0,
                               args=(y, X, etas, sigee, k2),
                               method=m, bounds=bounds,
                               options={'maxiter': 8000, 'ftol': 1e-9})
                if res.fun < best_fun:
                    best_res, best_fun = res, res.fun
            except Exception as e:
                warnings.warn(f"LIML optimization failed with {m}: {e}")

    if best_res is None:
        raise RuntimeError("All optimization attempts failed.")

    theta = best_res.x
    beta_hat = theta[:p]
    sig_ev_hat = theta[p:p+k2]
    log_sigma_v = theta[-2]
    log_lambda_c = theta[-1]

    cF = cho_factor(sigee, lower=False, check_finite=False)
    w = cho_solve(cF, sig_ev_hat, check_finite=False)
    correction = float(sig_ev_hat @ w)

    sigma_v_hat = float(np.exp(log_sigma_v))
    sigma_c2_hat = sigma_v_hat**2 - correction
    sigma_c_hat = float(np.sqrt(max(sigma_c2_hat, 1e-15)))
    lambda_c_hat = float(np.exp(log_lambda_c))
    sigma_u_hat = float(lambda_c_hat * sigma_c_hat)

    out = {
        'success': best_res.success,
        'message': best_res.message,
        'nit': getattr(best_res, 'nit', None),
        'nfev': getattr(best_res, 'nfev', None),
        'beta': beta_hat, 'sig_ev': sig_ev_hat,
        'sigma_v': sigma_v_hat, 'sigma_c': sigma_c_hat,
        'sigma_u': sigma_u_hat, 'lambda_c': lambda_c_hat,
        'w': w, 'theta': theta,
        'bounds': {'lower': lower, 'upper': upper}
    }
    return out

# ==============================
# Inference helpers (OPG)
# ==============================
def _pack_theta(beta, sig_ev, log_sigma_v, log_lambda_c):
    return np.concatenate([beta, sig_ev, [log_sigma_v, log_lambda_c]])

def _unpack_theta(theta, p, k2):
    beta = theta[:p]
    sig_ev = theta[p:p+k2]
    log_sigma_v = theta[-2]
    log_lambda_c = theta[-1]
    return beta, sig_ev, log_sigma_v, log_lambda_c

def ll_obs(theta, y, X, etas, sigee):
    p, k2 = X.shape[1], etas.shape[1]
    beta, sig_ev, lsv, llc = _unpack_theta(theta, p, k2)
    return lnL1_vec_fixed_sigee(beta, sig_ev, sigee, lsv, llc, y, X, etas)

def opg_vcov(theta_hat, y, X, etas, sigee, eps=1e-6):
    n = y.shape[0]
    k = theta_hat.size
    base = ll_obs(theta_hat, y, X, etas, sigee)  # (n,)
    G = np.empty((n, k))
    for j in range(k):
        th = theta_hat.copy()
        th[j] += eps
        fwd = ll_obs(th, y, X, etas, sigee)
        G[:, j] = (fwd - base) / eps
    OPG = G.T @ G
    try:
        vcov = np.linalg.inv(OPG)
    except np.linalg.LinAlgError:
        vcov = np.linalg.pinv(OPG)
    return vcov

def gradient_at(theta_hat, y, X, etas, sigee, eps=1e-6):
    k = theta_hat.size
    base = ll_obs(theta_hat, y, X, etas, sigee)
    g = np.zeros(k)
    for j in range(k):
        th = theta_hat.copy()
        th[j] += eps
        fwd = ll_obs(th, y, X, etas, sigee)
        g[j] = np.sum((fwd - base) / eps)  # gradient of total loglik
    return g

def lambda_c_profile(theta_hat, y, X, etas, sigee, grid=None):
    """Diagnostic profile: vary λ_c holding other params fixed (for quick sanity)."""
    p, k2 = X.shape[1], etas.shape[1]
    if grid is None:
        lam0 = float(np.exp(theta_hat[-1]))
        grid = np.array([lam0*0.25, lam0*0.5, lam0, lam0*2, lam0*4])
        grid = np.clip(grid, 0.01, 5.0)
    vals = []
    for lam in grid:
        th = theta_hat.copy()
        th[-1] = np.log(lam)
        vals.append(np.sum(ll_obs(th, y, X, etas, sigee)))
    return grid, np.array(vals)

# ==============================
# Real data preparation (paper mapping)
# ==============================
DATA_PATHS = [
    "/content/SPANISH DATASET.csv",     # CSV (uploaded)
    "/content/SPANISH DATASET.xlsx",     # Excel (fallback)
]

USECOLS = ['DEPREC', 'FARMEXP', 'MILK',
           'LABOR', 'COWS', 'FEED', 'LAND',
           'YEAR', 'COOP', 'PFEED', 'PMILK', 'LANDOWN',
           'COUNTY', 'ZONE']  # COUNTY/ZONE optional; code will handle if absent

def load_dataset() -> pd.DataFrame:
    path = None
    for p in DATA_PATHS:
        if os.path.exists(p):
            path = p
            break
    if path is None:
        raise FileNotFoundError("Could not find dataset. Checked: " + ", ".join(DATA_PATHS))

    if path.lower().endswith(".csv"):
        df = pd.read_csv(path)
    else:
        df = pd.read_excel(path)

    missing = [c for c in USECOLS if c not in df.columns]
    # Allow COUNTY/ZONE to be missing
    soft_missing = [c for c in missing if c in ('COUNTY', 'ZONE')]
    hard_missing = [c for c in missing if c not in ('COUNTY', 'ZONE')]
    if hard_missing:
        raise ValueError(f"Missing columns in data: {hard_missing}")

    keep_cols = [c for c in USECOLS if (c in df.columns)]
    df = df[keep_cols].copy()

    # Drop zero-roughage farms
    df = df[(df['DEPREC'] != 0) & (df['FARMEXP'] != 0)].copy()
    df.reset_index(drop=True, inplace=True)
    df['ROUGHAGE'] = df['FARMEXP'] + df['DEPREC']
    return df

def build_design_matrices(df: pd.DataFrame):
    # y = log(MILK) demeaned
    y = np.log(df['MILK'].values) - np.log(df['MILK'].values).mean()

    # Structural X: CONST + log(LABOR,COWS,FEED,LAND,ROUGHAGE) demeaned + YEAR dummies
    X_raw = df[['LABOR','COWS','FEED','LAND','ROUGHAGE']].copy()
    X_log = np.log(X_raw.values)
    X_log = X_log - X_log.mean(axis=0, keepdims=True)
    X_df = pd.DataFrame(X_log, columns=['LABOR','COWS','FEED','LAND','ROUGHAGE'])
    X_df.insert(0, 'CONST', 1.0)

    YEAR_dummy = pd.get_dummies(df['YEAR'], prefix='YEAR', drop_first=True, dtype=int)
    X_df = pd.concat([X_df, YEAR_dummy], axis=1)

    # Base instruments Z (A): CONST, LAND, prices, ownership, COOP, YEAR
    Z_parts = []
    Z_base = pd.DataFrame({'CONST': np.ones(len(df))})
    Z_parts.append(Z_base)

    Z_parts.append(X_df[['LAND']])  # LAND from X (already log-de-meaned)
    for col in ['PMILK','PFEED','LANDOWN']:
        if col in df.columns:
            Z_parts.append(df[[col]])

    # COOP dummies
    if 'COOP' in df.columns:
        COOP_dummy = pd.get_dummies(df['COOP'], prefix='COOP', drop_first=True, dtype=int)
        Z_parts.append(COOP_dummy)

    # YEAR dummies
    Z_parts.append(YEAR_dummy)

    # Optional geographic dummies if present
    if 'COUNTY' in df.columns:
        COUNTY_dummy = pd.get_dummies(df['COUNTY'], prefix='COUNTY', drop_first=True, dtype=int)
        Z_parts.append(COUNTY_dummy)
    if 'ZONE' in df.columns:
        ZONE_dummy = pd.get_dummies(df['ZONE'], prefix='ZONE', drop_first=True, dtype=int)
        Z_parts.append(ZONE_dummy)

    Z_df_A = pd.concat(Z_parts, axis=1)

    # (B) Squares of continuous instruments
    cont_cols = [c for c in ['LAND','PMILK','PFEED','LANDOWN'] if c in Z_df_A.columns]
    Z_sq = pd.DataFrame(index=Z_df_A.index)
    for c in cont_cols:
        Z_sq[c+'_SQ'] = Z_df_A[c]**2

    # (C) Selected interactions: LAND×prices, price×price, and LAND×LANDOWN
    Z_int = pd.DataFrame(index=Z_df_A.index)
    if 'LAND' in Z_df_A.columns and 'PMILK' in Z_df_A.columns:
        Z_int['LAND_PMILK'] = Z_df_A['LAND'] * Z_df_A['PMILK']
    if 'LAND' in Z_df_A.columns and 'PFEED' in Z_df_A.columns:
        Z_int['LAND_PFEED'] = Z_df_A['LAND'] * Z_df_A['PFEED']
    if 'PMILK' in Z_df_A.columns and 'PFEED' in Z_df_A.columns:
        Z_int['PMILK_PFEED'] = Z_df_A['PMILK'] * Z_df_A['PFEED']
    if 'LAND' in Z_df_A.columns and 'LANDOWN' in Z_df_A.columns:
        Z_int['LAND_LANDOWN'] = Z_df_A['LAND'] * Z_df_A['LANDOWN']

    # Final Z (A + B + C)
    Z_df = pd.concat([Z_df_A, Z_sq, Z_int], axis=1)

    # Endogenous block X2 (for NN): LABOR, COWS, FEED, ROUGHAGE
    X2_cols = ['LABOR','COWS','FEED','ROUGHAGE']
    X2 = X_df[X2_cols].values

    # Structural X: CONST, LABOR, COWS, FEED, LAND, ROUGHAGE + YEAR dummies
    struct_cols = ['CONST','LABOR','COWS','FEED','LAND','ROUGHAGE'] + list(YEAR_dummy.columns)
    X_struct = X_df[struct_cols].values
    X_struct_cols = struct_cols

    return y, X_struct, X_struct_cols, X2, Z_df.values, Z_df.columns

# ==============================
# Baseline (ElasticNet) helper
# ==============================
def crossfit_elasticnet_r2(Z: np.ndarray, X2: np.ndarray, K: int = 10, seed: int = BASE_SEED):
    """K-fold cross-fitted ElasticNetCV baseline per endogenous regressor."""
    n, _ = Z.shape
    k2 = X2.shape[1]
    preds = np.zeros_like(X2)
    nnz = np.zeros(k2, dtype=int)
    kf = KFold(n_splits=K, shuffle=True, random_state=seed)
    for j in range(k2):
        fold_nnz = []
        for tr, te in kf.split(np.arange(n)):
            Ztr, Zte = Z[tr], Z[te]
            xtr, xte = X2[tr, j], X2[te, j]
            Ztr_std, mu, sd = standardize_fit(Ztr)
            Zte_std = standardize_apply(Zte, mu, sd)
            encv = ElasticNetCV(
                l1_ratio=[0.2, 0.5, 0.8, 1.0],  # includes pure Lasso
                cv=5, random_state=seed, max_iter=10000
            )
            encv.fit(Ztr_std, xtr)
            preds[te, j] = encv.predict(Zte_std)
            fold_nnz.append(int(np.count_nonzero(encv.coef_)))
        nnz[j] = int(np.round(np.mean(fold_nnz)))
    r2s = [r2_score(X2[:, j], preds[:, j]) for j in range(k2)]
    return r2s, nnz.tolist()

# ==============================
# Main real-data run
# ==============================
def run_realdata(cfg: Optional[NNConfig] = None, rho_shrink: float = 0.15):
    if cfg is None:
        cfg = NNConfig()

    df = load_dataset()
    y, X_struct, Xcols, X2, Z_raw, Zcolnames = build_design_matrices(df)
    n = y.shape[0]

    # --- Report Z diagnostics (count, standardized condition number) ---
    print("\n== First-stage diagnostics ==")
    print(f"Z columns used (for OLS/Lasso): {Z_raw.shape[1]}")

    Zstd_full, muZ_full, sdZ_full = standardize_fit(Z_raw.astype(float))
    Zstd_noconst = Zstd_full[:, 1:] if Zstd_full.shape[1] > 1 else Zstd_full
    try:
        svals = np.linalg.svd(Zstd_noconst, full_matrices=False)[1]
        cond_Z = float(svals.max() / max(svals.min(), 1e-12))
    except Exception:
        cond_Z = np.nan
    print(f"Cond. number of Z (standardized, no CONST): {cond_Z:,.2f}")

    # --- PCA for NN (and optionally for OLS/EN) ---
    Z_for_nn = Z_raw.copy()
    m_kept = None
    if PCA_Z_FOR_NN:
        pca = PCA(n_components=None, svd_solver='full', random_state=BASE_SEED)
        Zp = pca.fit_transform(Zstd_full)
        cumvar = np.cumsum(pca.explained_variance_ratio_)
        m_kept = int(np.searchsorted(cumvar, PCA_VAR_KEEP) + 1)
        Z_for_nn = Zp[:, :m_kept]
        print(f"Z PCA -> components kept for NN: {m_kept} (variance ≥ {PCA_VAR_KEEP*100:.2f}%)")
    else:
        print("Z PCA disabled for NN.")

    Z_for_ols = Z_raw.copy()
    if PCA_Z_FOR_NN and PCA_Z_FOR_OLS_LASSO:
        # reuse the same PCA fit (on standardized Z)
        Z_for_ols = Zp[:, :m_kept]
        print("Using same PCA-compressed Z for OLS/ElasticNet baselines.")
    else:
        print("Baselines use raw Z.")

    # --- First-stage OLS baseline (no crossfit) ---
    ols_r2 = []
    for j in range(X2.shape[1]):
        lm = LinearRegression().fit(Z_for_ols, X2[:, j])
        ols_r2.append(float(lm.score(Z_for_ols, X2[:, j])))

    # --- ElasticNet (cross-fitted) baseline ---
    en_r2, en_nnz = crossfit_elasticnet_r2(Z_for_ols, X2, K=cfg.K, seed=cfg.seed)

    # --- NN-L2 with shrinkage Σ̂ηη ---
    eta_hat, Sigma_hat = crossfit_nn_l2(Z_for_nn, X2, cfg, rho_shrink=rho_shrink)
    X2_hat = X2 - eta_hat
    nn_r2 = [r2_score(X2[:, j], X2_hat[:, j]) for j in range(X2.shape[1])]

    print(f"OLS_R2_per_endog   (LABOR,COWS,FEED,ROUGHAGE): {_round_list(ols_r2)}")
    print(f"ElasticNet_R2_per_endog (LABOR,COWS,FEED,ROUGHAGE): {_round_list(en_r2)}")
    print(f"ElasticNet mean #nonzero per endog:                  {en_nnz}")
    print(f"NN_R2_per_endog    (LABOR,COWS,FEED,ROUGHAGE): {_round_list(nn_r2)}")


    # --- LIML L1 with fixed Σ̂ηη ---
    liml = estimate_LIML_L1_only(y, X_struct, eta_hat, Sigma_hat, fallback=True)

    print("\n== Optimizer ==")
    print("success :", liml['success'])
    print("message :", liml['message'])
    if liml.get('nit') is not None:
        print("nit     :", liml['nit'])
    if liml.get('nfev') is not None:
        print("nfev    :", liml['nfev'])

    # Bound hit check (for variance params)
    lb, ub = liml['bounds']['lower'], liml['bounds']['upper']
    log_sigma_v_hat = np.log(liml['sigma_v'])
    log_lambda_c_hat = np.log(liml['lambda_c'])
    print("\n== Bound check ==")
    print("hit log_sigma_v lower? ", np.isclose(log_sigma_v_hat, lb[-2]))
    print("hit log_sigma_v upper? ", np.isclose(log_sigma_v_hat, ub[-2]))
    print("hit log_lambda_c lower?", np.isclose(log_lambda_c_hat, lb[-1]))
    print("hit log_lambda_c upper?", np.isclose(log_lambda_c_hat, ub[-1]))

    # --- Structural betas ---
    betadf = pd.DataFrame(liml['beta'], index=Xcols, columns=['coef'])
    print("\n== Structural betas ==")
    print(betadf.round(4))

    # --- Variance components ---
    vardf = pd.DataFrame({
        'sigma_v': [liml['sigma_v']],
        'sigma_c': [liml['sigma_c']],
        'sigma_u': [liml['sigma_u']],
        'lambda_c': [liml['lambda_c']]
    })
    print("\n== Variance components ==")
    print(vardf.round(4))

    # --- Inference (OPG) ---
    p, k2 = X_struct.shape[1], eta_hat.shape[1]
    theta_hat = _pack_theta(liml['beta'], liml['sig_ev'],
                            np.log(liml['sigma_v']), np.log(liml['lambda_c']))
    vcov = opg_vcov(theta_hat, y, X_struct, eta_hat, Sigma_hat, eps=1e-6)
    se = np.sqrt(np.clip(np.diag(vcov), 0, np.inf))
    param_names = Xcols + [f"sig_ev[{j}]" for j in range(k2)] + ["log_sigma_v","log_lambda_c"]
    est = np.r_[liml['beta'], liml['sig_ev'], np.log(liml['sigma_v']), np.log(liml['lambda_c'])]
    ci_lo = est - 1.96*se
    ci_hi = est + 1.96*se
    inference_tbl = pd.DataFrame({"est": est, "se": se, "ci_lo": ci_lo, "ci_hi": ci_hi}, index=param_names)

    print("\n== Inference (OPG-based) ==")
    print(inference_tbl.round(4))

    # --- Technical Efficiency (TE) ---
    muci = eta_hat @ liml['w']
    r = y - X_struct @ liml['beta'] - muci
    te = te_from_r_sigma(r, liml['sigma_u'], liml['sigma_c'])
    te_summary = {
        "TE_mean": float(np.mean(te)),
        "TE_median": float(np.median(te)),
        "TE_p10": float(np.percentile(te, 10)),
        "TE_p25": float(np.percentile(te, 25)),
        "TE_p75": float(np.percentile(te, 75)),
        "TE_p90": float(np.percentile(te, 90))
    }
    print("\n== Technical Efficiency (TE) ==")
    print(pd.Series(te_summary).round(4))

    # --- Diagnostics ---
    ll = float(np.sum(ll_obs(theta_hat, y, X_struct, eta_hat, Sigma_hat)))
    yhat = X_struct @ liml['beta'] + muci
    sst = np.sum((y - y.mean())**2)
    sse = np.sum((y - yhat)**2)
    pseudo_R2 = 1.0 - sse / max(sst, 1e-12)
    grad = gradient_at(theta_hat, y, X_struct, eta_hat, Sigma_hat, eps=1e-6)
    grad_norm = float(np.linalg.norm(grad, ord=2))

    print("\n== Diagnostics ==")
    print(f"logLik (L1): {ll:.4f}")
    print(f"pseudo-R^2 (structural): {pseudo_R2:.4f}")
    print(f"||grad||_2 at optimum: {grad_norm:.3e}")

    # --- Σ̂ηη spectrum / identification check ---
    evals = np.linalg.eigvalsh(Sigma_hat)
    cond  = evals.max() / max(evals.min(), 1e-12)
    print("\n== Σ̂_{ηη} spectrum ==")
    print("eigenvalues:", np.round(evals, 6).tolist())
    print("condition number:", round(cond, 2))

    # --- λ_c profile (diagnostic) ---
    grid, vals = lambda_c_profile(theta_hat, y, X_struct, eta_hat, Sigma_hat)
    prof = pd.DataFrame({"lambda_c": grid, "logLik_L1": vals})
    print("\n== λ_c profile (diagnostic; other params held fixed) ==")
    print(prof.round(6).to_string(index=False))

    # Return everything in case you want to save/report
    out: Dict[str, Any] = {
        "betas": betadf,
        "variances": vardf,
        "inference": inference_tbl,
        "TE": te,
        "TE_summary": te_summary,
        "logLik_L1": ll,
        "pseudo_R2": pseudo_R2,
        "grad_norm": grad_norm,
        "Sigma_eta_eta": Sigma_hat,
        "Sigma_eigvals": evals,
        "Sigma_cond": cond,
        "first_stage_R2_OLS": ols_r2,
        "first_stage_R2_ElasticNet": en_r2,
        "ElasticNet_n_nonzero": en_nnz,
        "first_stage_R2_NN": nn_r2,
        "lambda_c_profile": prof,
        "liml": liml,
        "Xcols": Xcols,
        "Z_cond_std_no_const": cond_Z,
        "PCA_components_kept": m_kept if PCA_Z_FOR_NN else None,
    }
    return out

# ---------------- Main --------------------------
if __name__ == "__main__":
    print("\n=== Real-data NN-LIML (endog: LABOR, COWS, FEED, ROUGHAGE) ===")
    cfg = NNConfig()
    _ = run_realdata(cfg=cfg, rho_shrink=0.15)



=== Real-data NN-LIML (endog: LABOR, COWS, FEED, ROUGHAGE) ===

== First-stage diagnostics ==
Z columns used (for OLS/Lasso): 53
Cond. number of Z (standardized, no CONST): 90,747,661,659,658.56
Z PCA disabled for NN.
Baselines use raw Z.
OLS_R2_per_endog   (LABOR,COWS,FEED,ROUGHAGE): [0.5406, 0.7075, 0.6498, 0.6236]
ElasticNet_R2_per_endog (LABOR,COWS,FEED,ROUGHAGE): [0.4999, 0.6698, 0.605, 0.5872]
ElasticNet mean #nonzero per endog:                  [36, 39, 38, 34]
NN_R2_per_endog    (LABOR,COWS,FEED,ROUGHAGE): [0.636, 0.7362, 0.6658, 0.6426]

== Optimizer ==
success : True
message : Optimization terminated successfully
nit     : 212
nfev    : 5262

== Bound check ==
hit log_sigma_v lower?  False
hit log_sigma_v upper?  False
hit log_lambda_c lower? False
hit log_lambda_c upper? False

== Structural betas ==
             coef
CONST      0.0891
LABOR     -0.0395
COWS       0.9176
FEED       0.1715
LAND      -0.0910
ROUGHAGE   0.1377
YEAR_2000  0.0113
YEAR_2001  0.0079
YEAR_2002  0.0